<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

# Results Optiwise inverse dynamics

In [1]:
#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True
%load_ext kedro.ipython

[11/11/24 10:11:14] WARNING  Kedro extension was registered but couldn't find a Kedro project. Make  ]8;id=118601;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=987226;file:///home/maa/miniconda3/envs/phd/lib/python3.10/site-packages/kedro/ipython/__init__.py#40\40]8;;\
                             sure you run '%reload_kedro <project_root>'.                                          

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from vessel_manoeuvring_models.prime_system import PrimeSystem
from phd.visualization.plot_prediction import plot_total_force, plot_force_components, predict, plot_compare_model_forces, plot_parameter_contributions, group_parameters, joined_parameter_contributions, same_ylims
from vessel_manoeuvring_models.symbols import *
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify, remove_functions, prime
from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]
import sympy as sp
from sympy import Eq, symbols, latex
from phd.pipelines.load_wPCC.nodes import preprocess,zigzag_angle
from phd.pipelines.resimulate_with_autopilot.nodes import simulate_closed_loop, postprocess
from phd.visualization.plot_vct import plot_VCT, flat_keys

import paper
from paper import styles
from phd.visualization.units import units

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 import yaml                                                                                 │
│    7                                                                                             │
│    8 from vessel_manoeuvring_models.visualization.plot import track_plot, plot                   │
│ ❱  9 from phd.visualization.plot_ship import track_plots                                         │
│   10 from vessel_manoeuvring_models.prime_system import PrimeSystem                              │
│   11 from phd.visualization.plot_prediction import plot_total_force, plot_force_components, p    │
│   12 from vessel_manoeuvring_models.symbols import *                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'phd'

In [ ]:
ship_name = "optiwise"

In [ ]:
loaders = {
    "polynomial rudder" : catalog.load(f"{ship_name}.models_VCT_polynomial_rudder_MDL")['simple'],
    "semiempirical rudder" : catalog.load(f"{ship_name}.models_VCT_MDL")['semiempirical_covered_inertia'],
}

In [ ]:
models = {name:loader() for name,loader in loaders.items()}

## ID

In [ ]:
meta_data_all = catalog.load(f"{ship_name}.time_series_meta_data")
id_exists = [int(key) for key in catalog.load(f"{ship_name}.tests_ek_smooth1").keys()]
#id_exists = [int(key) for key in catalog.load(f"{ship_name}.tests_ek1").keys()]

ids = sorted(list(set(id_exists) & set(meta_data_all.index)))

meta_data_all = meta_data_all.loc[ids].copy()
#ids = [int(key) for key in catalog.load("wPCC.tests_ek_smooth1").keys()]

mask = ((meta_data_all['test_type'] == 'zigzag') & (meta_data_all['ship_speed'] > 0.9))

meta_datas = meta_data_all.loc[mask].copy()

ids = [
    23590,
    23591,
    23594,
    23596,
]
meta_datas = meta_datas.loc[ids].copy()
meta_datas['angle'] = [
    -10,
    10,
    -20,
    20,
]

In [ ]:
for id, meta_data in meta_datas.iterrows():

    df_EKF_smooth = catalog.load(f"{ship_name}.tests_ek_smooth1")[str(id)]()
    df_EKF_smooth['U'] = np.sqrt(df_EKF_smooth['u']**2 + df_EKF_smooth['v']**2)
    #df_EKF_smooth['thrust'] = df_EKF_smooth['thrust_port'] + df_EKF_smooth['thrust_stbd']
    #data = load(id=id, catalog=catalog, smooth=False)
    #data = filter(data,cutoff=1)
    #data = data.iloc[100:-100].copy()

    if id==22774:
        #data=data.loc[0:70].copy()
        df_EKF_smooth = df_EKF_smooth.loc[0:70].copy()


    fig = plot_compare_model_forces(model=models['polynomial rudder'], models=models, data=df_EKF_smooth, styles=styles, keys=['N_D','Y_D'], delta_corners=False);
    #fig = plot_compare_model_forces(model=models["ID small"], models=models, data=df_EKF_smooth, styles=styles, keys=['N_VCT','Y_VCT','N_H','N_R'], delta_corners=False);
    
    #fig.axes[-1].legend(loc='upper center', bbox_to_anchor=(0.05, 2.2), ncol=1)
    #fig.axes[0].legend(loc='upper center', bbox_to_anchor=(0.25, 2.2), ncol=3)
    fig.axes[1].legend(loc='upper center', bbox_to_anchor=(0.45, 1.30), ncol=3)
    #fig.axes[1].legend(bbox_to_anchor=(0.7, 1.5), ncol=5)
    #fig.axes[1].legend(ncol=3)
    
    for ax in fig.axes[1:5]:
        ylims = ax.get_ylim()
        ylims = [min([ylims[0],-30]), max([ylims[1],30]),]
        ax.set_ylim(ylims)
    
    direction = "port" if meta_data['angle'] > 0 else "stbd"
    name = f"{meta_data['test_type']} {meta_data['angle1']:0.0f}/{meta_data['angle1']:0.0f} {direction}"
    fig.suptitle(name)

    file_name = name.replace("/","_")
    paper.save_fig(fig, f"{file_name}.svg")


In [ ]:


id=23594
df_EKF_smooth = catalog.load(f"{ship_name}.tests_ek_smooth1")[str(id)]()
df_EKF_smooth['U'] = np.sqrt(df_EKF_smooth['u']**2 + df_EKF_smooth['v']**2)

fig,ax=plt.subplots()
models['semiempirical rudder'].forces_from_motions(df_EKF_smooth).plot(y='Y_D',ax=ax)
predict(model=models['semiempirical rudder'], data=df_EKF_smooth).plot(y='Y_D', ax=ax)

fig,ax=plt.subplots()
models['semiempirical rudder'].forces_from_motions(df_EKF_smooth).plot(y='N_D',ax=ax)
predict(model=models['semiempirical rudder'], data=df_EKF_smooth).plot(y='N_D', ax=ax)

## Closed loop simulation

In [ ]:
for angle, group in meta_datas.groupby(by='angle1'):

    fig,ax=plt.subplots()
    include_label=True
    
    for id, meta_data in group.iterrows():
    
        df_EKF_smooth = catalog.load(f"{ship_name}.tests_ek_smooth1")[str(id)]()
        df_EKF_smooth['U'] = np.sqrt(df_EKF_smooth['u']**2 + df_EKF_smooth['v']**2)
    
        dataframes = {'Experiment':df_EKF_smooth}
        
        for model_name, model in models.items():
        
            data = preprocess(df_EKF_smooth,ship_data=model.ship_parameters)
            #angle = zigzag_angle(data_MDL=data)
        
            df_simulation = simulate_closed_loop(model=model,data=data)
        
            direction = "port" if meta_data['angle'] > 0 else "stbd"
            name = f"{meta_data['test_type']} {meta_data['angle1']:0.0f}/{meta_data['angle1']:0.0f} {direction} {model_name}"
        
            dataframes[model_name] = df_simulation
    
        track_plots(dataframes, lpp=model.ship_parameters['L'], beam=model.ship_parameters['B'], include_wind=False, styles=styles, ax=ax, include_label=include_label);
        fig = ax.get_figure()
        direction = "port" if meta_data['angle'] > 0 else "stbd"
        name = f"{meta_data['test_type']} {meta_data['angle1']:0.0f}/{meta_data['angle1']:0.0f}"
        fig.suptitle(name)
        file_name = name.replace("/","_")
        paper.save_fig(fig, f"track plot {file_name}.svg")
        
        fig = plot(dataframes, keys=['psi','beta','V','delta'], units=units, styles=styles, ncols=2, include_label=include_label);
        name = f"{meta_data['test_type']} {meta_data['angle1']:0.0f}/{meta_data['angle1']:0.0f} {direction}"
        fig.suptitle(name)
        file_name = name.replace("/","_")
        paper.save_fig(fig, f"closed loop {file_name}.svg")
        
        
        include_label=False

In [ ]:
styles